In [1]:
import numpy as np
import torch
import os
import pandas as pd
import torch_geometric
import pickle

/home/icb/vnicolas.goedert/miniconda3/envs/RFAM/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
rfam_dir = "../RNARepLearn/rfam/data/raw/processed/release-14.8"
rfam_id = "RF00008"

In [149]:
class SingleMaskedRfamDataset(torch_geometric.data.Dataset): 
    
    ###Contains single RFAM family with masked sequence
    
    def __init__(self, rfam_dir, rfam_id, transform=None, pre_transform=None, pre_filter=None):
        self.dir = rfam_dir
        self.id = rfam_id
        super().__init__(rfam_dir, transform, pre_transform, pre_filter)
    
    @property
    def raw_file_names(self):
        return(os.path.join(self.dir, self.id, self.id+".npy"))
    
    @property
    def processed_file_names(self):
        return os.listdir(os.path.join(self.dir,self.id,"pt"))
    
    
    def one_hot_encode(self, seq):
        nuc_d = {0:[1.0,0.0,0.0,0.0],
                 1:[0.0,1.0,0.0,0.0],
                 2:[0.0,0.0,1.0,0.0],
                 3:[0.0,0.0,0.0,1.0]}
        vec=np.array([nuc_d[x] for x in seq])
        return vec
    
    def generate_edges(self,seq_len,bpp):
        X = np.zeros((seq_len,seq_len))
        X[np.triu_indices(X.shape[0], k = 1)] = bpp
        X = X+X.T
        df = pd.DataFrame(X)
        np.fill_diagonal(df.values, np.nan)
        adf = df.stack().reset_index()
        adf = adf.rename(columns={"level_0":"A","level_1":"B",0:"weight"})
        return (adf.loc[adf["weight"]!=0.0])[["A","B"]].to_numpy()
    
    
    def process(self):
        if not os.path.exists(os.path.join(self.dir, self.id,"pt")):
            os.makedirs(os.path.join(self.dir, self.id,"pt"))
        with open(self.raw_file_names,"rb") as f:
            n = 0
            while True:
                try: 
                    array = np.load(f, allow_pickle=True)
                    
                    
                    ## seq (node features one-hot encoded sequence) and classes (node classes) are essentially the same right now. Just for proof of concept
                    ## TODO replace with significant features (e.g. position/order in sequence)
                    
                    seq = torch.tensor(self.one_hot_encode(array.item()['seq_int']))
                    classes = torch.tensor(array.item()['seq_int'])
                    
                    struc = torch.tensor(self.generate_edges(len(array.item()['seq_int']),array.item()['bpp']))
                    rfam_id = array.item()['id'].replace("/","_")
                    torch.save({"seq":seq, "edges":struc, "rfam":array.item()['rfam_id'],"classes":classes ,"id":rfam_id}, os.path.join(self.dir, self.id,"pt",rfam_id+".pt"))
                    
                except pickle.UnpicklingError:
                    
                    break
            f.close()
        
        
    def __len__(self):
        return len(self.processed_file_names)
    
    def __getitem__(self, index):
        self.processed_file_names[index]
        data = torch.load(os.path.join(self.dir,self.id,"pt",self.processed_file_names[index]))
        return torch_geometric.data.Data(x=data["seq"],edge_index=data["edges"].t().contiguous(),y=data["classes"],rfam=data["rfam"],ID=data["id"])
        
            

In [150]:
rfam_dataset = SingleMaskedRfamDataset(rfam_dir,rfam_id)

Processing...
Done!


In [165]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(rfam_dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, 4)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [166]:
## Set up mask
import random
train_mask = torch.tensor([random.randrange(100) < 30 for i in range(rfam_dataset[0].num_nodes)])
test_mask = torch.tensor([random.randrange(100) < 30 for i in range(rfam_dataset[0].num_nodes)])

In [190]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
model = model.double()
data = rfam_dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[train_mask], data["y"][train_mask].long())
    loss.backward()
    optimizer.step()

In [193]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[test_mask] == data.y[test_mask]).sum()
acc = int(correct) / int(test_mask.sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 1.0000
